In [ ]:
import glob
from osgeo import gdal, ogr, osr
import rasterio as rio
import numpy as np
from pathlib import Path
import geopandas as gpd
from rasterstats import zonal_stats

In [ ]:
dif_imgs = glob.glob('/filserver/user/imagery/results/difference_*2021_2023.tiff')
dif_imgs

In [ ]:
def normalize_2(band):
    band_min, band_max = (band.min(), band.max())
    print(band_min, band_max)
    return ((band-band_min)/((band_max - band_min))) * 2 - 1

for in_path in dif_imgs:
    print(in_path)
    norm_path = f"/filserver/user/imagery/results/{Path(in_path).stem}_norm.tiff"

    src_ds = rio.open( in_path )
    meta = src_ds.meta
    srcband = src_ds.read(1)
    norm_srcband = normalize_2(np.nan_to_num(srcband))
    with rio.open(norm_path, 'w', **meta) as dest:
        dest.write(norm_srcband,1)
    dest.close()

In [ ]:
norm_dif_imgs = glob.glob('/filserver/user/imagery/results/difference_*_2021_2023_norm.tiff')
norm_dif_imgs

In [ ]:
for in_path in norm_dif_imgs:
    out_path = f"/filserver/frruserov/imagery/results/{Path(in_path).stem}.geojson"
    # tmp_tiff = '/filserver/user/imagery/results/tmp.tiff'

    src_ds = gdal.Open( in_path )
    srcband = src_ds.GetRasterBand(1)

    dst_layername = 'polygon'
    drv = ogr.GetDriverByName("GeoJSON")
    dst_ds = drv.CreateDataSource( out_path )

    sp_ref = osr.SpatialReference()
    sp_ref.SetFromUserInput('EPSG:25832')

    dst_layer = dst_ds.CreateLayer(dst_layername, srs = sp_ref )

    fld = ogr.FieldDefn("DN", ogr.OFTInteger)
    dst_layer.CreateField(fld)
    dst_field = dst_layer.GetLayerDefn().GetFieldIndex("DN")

    gdal.Polygonize( srcband, None, dst_layer, dst_field, [], callback=None )

    del src_ds
    del dst_ds
    # os.remove(tmp_tiff)

    gdf = gpd.read_file(out_path)
    selected_polygons = gdf[gdf['DN'] != 0]
    large_polygons = selected_polygons[selected_polygons.area >= 400]

    stats = zonal_stats(large_polygons, in_path, stats="mean", geojson_out=True)
    result_gdf = gpd.GeoDataFrame.from_features(stats, crs=large_polygons.crs)
    result_gdf.to_file(out_path, driver='GeoJSON')
    print(out_path)